In [2]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.model_selection import train_test_split

In [ ]:
class SBS():
    def __init__(self, classifier, k_features, current_score, test_size=.25, random_state=1):
        self.current_score = current_score
        self.classifier = classifier
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state
    
    def fit(self, X, y):
        X_train, X_test, y_train, y_test = \ 
        train_test_split (
            X, y, test_size=self.test_size, 
            stratify=y, random_state=self.random_state
        )

        dim = X_train.shape[1]
        self.indices = tuple(range(dim))
        self.subsets = [self.indices]
        score = self.get_score(X_train, X_test, y_train, y_test, self.indices)

        self.scores = [score]

        while dim > self.k_features:
            scores = []
            subsets = []

            for p in combinations(self.indices, r=dim-1):
                score = self.get_score(
                    X_train, X_test, y_train, y_test, p)
                scores.append(score)
                subsets.append(p)

            best_acc_index = np.argmax(scores)
            self.indices = subsets[best_acc_index]
            

    def get_score(X_train, X_test, y_train, y_test, col_indices):
        self.classifier.fit(X_train.iloc[:, col_indices], y_train)
        self.current_score = self.classifier.score(
            X_test.iloc[:, col_indices], y_test)
        return self.current_score